# DataFrames: Read and Write Data
     
Dask Dataframes can read and store data in many of the same formats as Pandas dataframes.  In this example we read and write data with the popular CSV and Parquet formats, and discuss best practices when using these formats.

In [1]:
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/0eEsIA0O1iE?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

/home/travis/miniconda/envs/test/lib/python3.8/site-packages/IPython/core/display.py:717: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [2]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client

Client Scheduler: inproc://10.20.1.11/12569/1 Dashboard: http://10.20.1.11:8787/status,Cluster Workers: 1 Cores: 4 Memory: 2.00 GB


## Create artificial dataset

First we create an artificial dataset and write it to many CSV files.

You don't need to understand this section, we're just creating a dataset for the rest of the notebook.

In [3]:
import dask
df = dask.datasets.timeseries()
df

,id,name,x,y
npartitions=30,,,,
2000-01-01,int64,object,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2000-01-30,...,...,...,...
2000-01-31,...,...,...,...


In [4]:
import os
import datetime

if not os.path.exists('data'):
    os.mkdir('data')

def name(i):
    """ Provide date for filename given index
    
    Examples
    --------
    >>> name(0)
    '2000-01-01'
    >>> name(10)
    '2000-01-11'
    """
    return str(datetime.date(2000, 1, 1) + i * datetime.timedelta(days=1))
    
df.to_csv('data/*.csv', name_function=name);

## Read CSV files

We now have many CSV files in our data directory, one for each day in the month of January 2000.  Each CSV file holds timeseries data for that day.  We can read all of them as one logical dataframe using the `dd.read_csv` function with a glob string.

In [5]:
!ls data/*.csv | head

data/2000-01-01.csv
data/2000-01-02.csv
data/2000-01-03.csv
data/2000-01-04.csv
data/2000-01-05.csv
data/2000-01-06.csv
data/2000-01-07.csv
data/2000-01-08.csv
data/2000-01-09.csv
data/2000-01-10.csv


In [6]:
!head data/2000-01-01.csv

timestamp,id,name,x,y
2000-01-01 00:00:00,954,Charlie,-0.7720471689611406,0.6406211770972339
2000-01-01 00:00:01,1018,Victor,-0.9528417493803996,-0.7265462025243281
2000-01-01 00:00:02,999,Ray,0.4544630486655328,-0.6504895540404549
2000-01-01 00:00:03,1009,George,0.4697043382657822,-0.7019323134682958
2000-01-01 00:00:04,977,Jerry,0.8112194108602526,0.7897121993813936
2000-01-01 00:00:05,1000,Zelda,0.17261918184305203,0.06115571488709004
2000-01-01 00:00:06,950,Ursula,0.4849541901908141,-0.2675541410388933
2000-01-01 00:00:07,975,Edith,-0.4722562783787436,0.5464531845944569
2000-01-01 00:00:08,967,Edith,0.6853587913231043,-0.25953638750801544


In [7]:
!head data/2000-01-30.csv

timestamp,id,name,x,y
2000-01-30 00:00:00,1012,Frank,0.592018341967588,0.501396556401217
2000-01-30 00:00:01,976,Frank,-0.48507453958221025,-0.6070660020462681
2000-01-30 00:00:02,964,Oliver,0.6742916132513597,-0.6026316552290334
2000-01-30 00:00:03,990,Tim,-0.5059193111879889,0.18065976886428947
2000-01-30 00:00:04,954,Jerry,-0.6840693490069598,0.29243912659443705
2000-01-30 00:00:05,1017,Ray,0.0034857595949182585,0.24653037065870143
2000-01-30 00:00:06,997,Laura,-0.42466989979815817,0.003092154994748242
2000-01-30 00:00:07,1003,Alice,-0.36133558817783595,-0.08721324040410905
2000-01-30 00:00:08,986,Charlie,-0.11114758713512551,-0.589266105465855


We can read one file with `pandas.read_csv` or many files with `dask.dataframe.read_csv`

In [8]:
import pandas as pd

df = pd.read_csv('data/2000-01-01.csv')
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,954,Charlie,-0.772047,0.640621
1,2000-01-01 00:00:01,1018,Victor,-0.952842,-0.726546
2,2000-01-01 00:00:02,999,Ray,0.454463,-0.650490
3,2000-01-01 00:00:03,1009,George,0.469704,-0.701932
4,2000-01-01 00:00:04,977,Jerry,0.811219,0.789712


In [9]:
import dask.dataframe as dd

df = dd.read_csv('data/2000-*-*.csv')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,object,int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [10]:
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,954,Charlie,-0.772047,0.640621
1,2000-01-01 00:00:01,1018,Victor,-0.952842,-0.726546
2,2000-01-01 00:00:02,999,Ray,0.454463,-0.650490
3,2000-01-01 00:00:03,1009,George,0.469704,-0.701932
4,2000-01-01 00:00:04,977,Jerry,0.811219,0.789712


## Tuning read_csv

The Pandas `read_csv` function has *many* options to help you parse files.  The Dask version uses the Pandas function internally, and so supports many of the same options.  You can use the `?` operator to see the full documentation string.

In [11]:
pd.read_csv?

In [12]:
dd.read_csv?

In this case we use the `parse_dates` keyword to parse the timestamp column to be a datetime.  This will make things more efficient in the future.  Notice that the dtype of the timestamp column has changed from `object` to `datetime64[ns]`.

In [13]:
df = dd.read_csv('data/2000-*-*.csv', parse_dates=['timestamp'])
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


## Do a simple computation

Whenever we operate on our dataframe we read through all of our CSV data so that we don't fill up RAM.  This is very efficient for memory use, but reading through all of the CSV files every time can be slow.

In [14]:
%time df.groupby('name').x.mean().compute()

CPU times: user 4.74 s, sys: 596 ms, total: 5.33 s
Wall time: 3.48 s


name
Alice      -0.000208
Bob        -0.003139
Charlie     0.000773
Dan        -0.001381
Edith      -0.000733
Frank       0.002132
George      0.002228
Hannah     -0.000271
Ingrid      0.000966
Jerry      -0.000379
Kevin       0.000334
Laura       0.003352
Michael    -0.001106
Norbert    -0.003203
Oliver     -0.001114
Patricia    0.000278
Quinn       0.001750
Ray         0.002947
Sarah       0.000078
Tim        -0.000716
Ursula     -0.002091
Victor     -0.000152
Wendy       0.000176
Xavier     -0.000101
Yvonne     -0.002060
Zelda       0.000504
Name: x, dtype: float64

## Write to Parquet

Instead, we'll store our data in Parquet, a format that is more efficient for computers to read and write.

In [15]:
df.to_parquet('data/2000-01.parquet', engine='pyarrow')

In [16]:
!ls data/2000-01.parquet/

_common_metadata  part.15.parquet  part.22.parquet  part.2.parquet
_metadata	  part.16.parquet  part.23.parquet  part.3.parquet
part.0.parquet	  part.17.parquet  part.24.parquet  part.4.parquet
part.10.parquet   part.18.parquet  part.25.parquet  part.5.parquet
part.11.parquet   part.19.parquet  part.26.parquet  part.6.parquet
part.12.parquet   part.1.parquet   part.27.parquet  part.7.parquet
part.13.parquet   part.20.parquet  part.28.parquet  part.8.parquet
part.14.parquet   part.21.parquet  part.29.parquet  part.9.parquet


## Read from Parquet

In [17]:
df = dd.read_parquet('data/2000-01.parquet', engine='pyarrow')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [18]:
%time df.groupby('name').x.mean().compute()

CPU times: user 1.51 s, sys: 119 ms, total: 1.63 s
Wall time: 1.13 s


name
Alice      -0.000208
Bob        -0.003139
Charlie     0.000773
Dan        -0.001381
Edith      -0.000733
Frank       0.002132
George      0.002228
Hannah     -0.000271
Ingrid      0.000966
Jerry      -0.000379
Kevin       0.000334
Laura       0.003352
Michael    -0.001106
Norbert    -0.003203
Oliver     -0.001114
Patricia    0.000278
Quinn       0.001750
Ray         0.002947
Sarah       0.000078
Tim        -0.000716
Ursula     -0.002091
Victor     -0.000152
Wendy       0.000176
Xavier     -0.000101
Yvonne     -0.002060
Zelda       0.000504
Name: x, dtype: float64

## Select only the columns that you plan to use

Parquet is a column-store, which means that it can efficiently pull out only a few columns from your dataset.  This is good because it helps to avoid unnecessary data loading.

In [19]:
%%time
df = dd.read_parquet('data/2000-01.parquet', columns=['name', 'x'], engine='pyarrow')
df.groupby('name').x.mean().compute()

CPU times: user 1.34 s, sys: 69.2 ms, total: 1.41 s
Wall time: 997 ms


name
Alice      -0.000208
Bob        -0.003139
Charlie     0.000773
Dan        -0.001381
Edith      -0.000733
Frank       0.002132
George      0.002228
Hannah     -0.000271
Ingrid      0.000966
Jerry      -0.000379
Kevin       0.000334
Laura       0.003352
Michael    -0.001106
Norbert    -0.003203
Oliver     -0.001114
Patricia    0.000278
Quinn       0.001750
Ray         0.002947
Sarah       0.000078
Tim        -0.000716
Ursula     -0.002091
Victor     -0.000152
Wendy       0.000176
Xavier     -0.000101
Yvonne     -0.002060
Zelda       0.000504
Name: x, dtype: float64

Here the difference is not that large, but with larger datasets this can save a great deal of time.

## Learn more

http://dask.pydata.org/en/latest/dataframe-create.html